# Analyses performed in the Manuscript

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import utils as utl
import matplotlib.pyplot as plt

from matplotlib import cm
from matplotlib import colors
import matplotlib.ticker as mticker
from scipy.interpolate import UnivariateSpline, LSQUnivariateSpline
from scipy.ndimage import gaussian_filter1d   
from functools import partial

import gc

import warnings
import re
import importlib

import data_processing as dp
from data_processing import didx

import common_plot_parameters as cpprm

idx = pd.IndexSlice

# Dark-Light transition

In [ ]:
# import raw data
EXPERIMENT_PATH="data/20250227_DarkLightTransition_25umol"
FILEPATHS = [x for x in Path(EXPERIMENT_PATH).glob("*") if x.is_dir() and not x.name.startswith("_")]

ojip, levels = dp.load_data(FILEPATHS)


# Load the plot parameters
spec=importlib.util.spec_from_file_location("plot_parameters",Path(EXPERIMENT_PATH)/"plot_parameters.py")
pprm = importlib.util.module_from_spec(spec)
spec.loader.exec_module(pprm)
plot_parameters=pprm.plot_parameters

## Get the main points of the OJIP curves

In [ ]:
ojip_features, ojip_features_meansd = dp.get_ojip_features(ojip)

## Double normalize the data

In [ ]:
ojip_norm = dp.normalize_ojip(ojip, ojip_features=ojip_features)

## Get the ojip points

In [ ]:
# Get the ojip points
with warnings.catch_warnings(record=False) as caught_warnings:
    warnings.simplefilter("ignore") 
    ojip_points_res = utl.determine_OJIP_points(ojip_norm, **dp.feature_finding_options)

ojip_points = ojip_points_res["points"]

In [ ]:
# Calculate the mean ojip curves
ojip_norm_meansd = pd.concat({
    "mean":ojip_norm.T.groupby(ojip_norm.columns.names[:-1]).mean(),
    "sd":ojip_norm.T.groupby(ojip_norm.columns.names[:-1]).std()
    }, names=["Measure"]).T

VJ_values_meansd = pd.concat({
    "mean":VJ_values.T.groupby(VJ_values.index.names[:-1]).mean(),
    "sd":VJ_values.T.groupby(VJ_values.index.names[:-1]).std()
    }, names=["Measure"]).T

## Paper figure

In [ ]:
LIGHTPHASES_PATH = Path(EXPERIMENT_PATH) / "light_phases.csv"
if LIGHTPHASES_PATH.is_file():
    light_phases = pd.read_csv(LIGHTPHASES_PATH)
else:
    light_phases = None

# Get the base plot "AL {treatment:d}s"
fig, axes = utl.get_base_plot(
    ojip_norm,
    ojip_norm_meansd,
    ojip_points,
    VJ_timing,
    VJ_values,
    VJ_values_meansd,
    levels,
    plot_replicates = False,
    use_colorbar = False,
    mark_sampled = True,
    cmap = cm.coolwarm,
    light_phases=light_phases,
    **plot_parameters,
    **cpprm.common_plot_parameters_main,
)

for ext in cpprm.plot_format:
    fig.savefig(Path("figures")/f"{EXPERIMENT_PATH.split("/")[1]}.{ext}", bbox_inches="tight")

## SI: Raw OJIP and P-timing

In [ ]:
# Make the FP-finding range slimmer for Syn
# Remove FP timing from default options
feature_finding_options = dp.feature_finding_options.copy()
feature_finding_options.pop("FP_time_min")

ojip_points_raw_res={}

# Identify the points for both strains and use a slimmer detection range for Syn
for strain in levels["strains"]:
    # Get the ojip pints
    with warnings.catch_warnings(record=False) as caught_warnings:
        warnings.simplefilter("ignore") 
        ojip_points_raw_res[strain] = utl.determine_OJIP_points(ojip.loc[:,didx(strain=strain)],
                                        return_derivatives=True,
                                        return_fits=True,
                                        FP_time_min=40 if strain=="Chlo" else 100,
                                        choose_method="closest",
                                        FJ_time_exp=2,
                                        FI_time_exp=30,
                                        FP_time_exp=100 if strain=="Chlo" else 300,
                                        **feature_finding_options)

ojip_points_raw = pd.concat([ojip_points_raw_res[strain]["points"] for strain in levels["strains"]], axis=0)

In [ ]:
ojip_meansd = pd.concat({
    "mean":ojip.T.groupby(ojip.columns.names[:-1]).mean(),
    "sd":ojip.T.groupby(ojip.columns.names[:-1]).std()
    }, names=["Measure"]).T

FP_values = ojip_points_raw[("grad2-min", "FP_value")]
FP_values_meansd = pd.concat({
    "mean":FP_values.T.groupby(FP_values.index.names[:-1]).mean(),
    "sd":FP_values.T.groupby(FP_values.index.names[:-1]).std()
    }, names=["Measure"]).T

FP_timing = ojip_points_raw[("grad2-min", "FP_time")]
FP_timing_meansd = pd.concat({
    "mean":FP_timing.T.groupby(FP_timing.index.names[:-1]).mean(),
    "sd":FP_timing.T.groupby(FP_timing.index.names[:-1]).std()
    }, names=["Measure"]).T

In [ ]:
LIGHTPHASES_PATH = Path(EXPERIMENT_PATH) / "light_phases.csv"
if LIGHTPHASES_PATH.is_file():
    light_phases = pd.read_csv(LIGHTPHASES_PATH)
else:
    light_phases = None

# Get the base plot "AL {treatment:d}s"
fig, axes = utl.get_base_plot(
    ojip,
    ojip_meansd,
    ojip_points_raw,
    None,
    FP_values,
    FP_timing_meansd,
    levels,
    plot_replicates = False,
    use_colorbar = False,
    mark_sampled = True,
    cmap = cm.coolwarm,
    light_phases=light_phases,
    variance_sleeve_alpha=0.3,
    right_column_y_label=r"F$_{\mathrm{P}}$ timing (ms)",
    left_column_y_label = "Fluorescence (V)",
    right_column_mark_zero=True,
    point_x_selector=("grad2-min", "FP_time"),
    point_y_selector=("grad2-min", "FP_value"),
    point_label="Identified FP",
    **plot_parameters,
    **cpprm.common_plot_parameters_SI,
)

for ax in axes[:,:-1].flatten():
    ax.set_ylim(0)

# for ax in axes[:,-1]:
#     if ax.get_ylim()[1] > 500:
#         ax.set_ylim(*ax.get_ylim())
#         ax.axhspan(550, 700, color="k", alpha=0.2)
#         ax.text(0.55, 0.78, r"F$_{\mathrm{P}}$ likely not reached", transform=ax.transAxes)

for ext in cpprm.plot_format:
    fig.savefig(Path("figures")/f"{EXPERIMENT_PATH.split("/")[1]}_SI.{ext}", bbox_inches="tight")